# Introduction

Welcome to the Snowplow Signals Quickstart! This guide will walk you through the steps to set up and use Snowplow Signals to define attributes, create views, and retrieve user attributes. By the end of this guide, you'll have a working setup to personalize your applications using Snowplow Signals.

1. Install and Set Up the SDK: Get started with the Snowplow Signals SDK.
2. Define an Attribute: Learn how to define user behavior attributes.
3. Create and Test a View: Combine attributes into a view and test it.
4. Deploy a View: Make your view live in the Signals API.
5. Retrieve Attributes: Fetch user attributes from the API.


# Installation and setup

In [ ]:
%pip install snowplow-signals

Processing ./snowplow_signals-0.1.0-py3-none-any.whl
snowplow-signals is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


To connect to your deployment, initialize the Signals object with your API credentials:


In [ ]:
from snowplow_signals import Signals
from google.colab import userdata

signals = Signals(
    api_url=userdata.get('SP_API_URL'),
    api_key=userdata.get('SP_API_KEY'),
    api_key_id=userdata.get('SP_API_KEY_ID'),
    org_id=userdata.get('SP_ORG_ID'),
)


# Define an Attribute
An Attribute represents a specific fact about a user's behavior. For example, you can define an attribute to count the number of page_view events a user has made.

In [ ]:
from snowplow_signals import Attribute, Event

page_view_count = Attribute(
    name="page_view_count",
    type="int32",
    events=[
        Event(
            vendor="com.snowplowanalytics.snowplow",
            name="page_view",
            version="1-0-0",
        )
    ],
    aggregation="counter"
)

You can also create more complex attributes by adding Criteria to filter events. For example:

In [ ]:
from snowplow_signals import Attribute, Event, Criteria, Criterion

products_added_to_cart_attribute = Attribute(
    name="products_added_to_cart",
    type="string_list",
    events=[
        Event(
            vendor="com.snowplowanalytics.snowplow.ecommerce",
            name="snowplow_ecommerce_action",
            version="1-0-2",
        )
    ],
    aggregation="unique_list",
    property="contexts_com_snowplowanalytics_snowplow_ecommerce_product_1[0].name",
    filter=Criteria(
        all=[
            Criterion(
                property="unstruct_event_com_snowplowanalytics_snowplow_ecommerce_snowplow_ecommerce_action_1:type",
                operator="=",
                value="add_to_cart",
            ),
        ],
    ),
)

# Create and test a View
A View is a collection of attributes that share a common entity (e.g., session or user). Here's how to create a view with the attributes we defined earlier:



In [ ]:
from snowplow_signals import View, domain_sessionid

my_attribute_view = View(
    name="my_attribute_view",
    version=1,
    entity=domain_sessionid,
    attributes=[
        products_added_to_cart_attribute,
    ],
    owner="user@company.com",
)

Before deploying the view, you can test it on the Atomic events table from the past hour:



In [ ]:
data = signals.test(
    view=my_attribute_view,
)
data

,domain_sessionid,products_added_to_cart
0,13b647f26d0e20e38df175cbfb23b3bd913305a95c4158...,[]
1,964b10e7c55f52960512d5986802e589461861269192a6...,[]
2,cb01a438f06a1273ecd47b37b181e73f9d1db67bdb5ef5...,[]
3,dbb02517fafd6daa4e62b63d39b51d30b82635ce6316af...,[]
4,825f69a5dd4f19405d2cb71f9acea56e22f54e09a44909...,[]
5,87313dfaff2a36536a599da74c703c6c010f762c641d59...,[]
6,8c30474123972169f64000bb2766fa2911f39c8eba8536...,[]
7,b7ba43a3b1c1b18caa86bf7ca5bfe4f33e86258304944f...,[]
8,9037b549897104ebb4847525e7d20092b5bd6caef5f66d...,[]
9,475a0eb55dc8186cc5c94957a02ed682f735ba9dac6f22...,[]


# Deploy a View to Signals
Once you're satisfied with the View, deploy it to the API using the apply method:



In [ ]:
signals.apply([my_attribute_view])


[ViewOutput(name='my_attribute_view', version=1, entity=LinkEntity(name='session'), ttl=None, batch_source=None, online=True, description=None, tags=None, owner=None, fields=[], attributes=[AttributeOutput(name='products_added_to_cart', description=None, type='string_list', tags=None, events=[Event(name='snowplow_ecommerce_action', vendor='com.snowplowanalytics.snowplow.ecommerce', version='1-0-2')], aggregation='unique_list', property='contexts_com_snowplowanalytics_snowplow_ecommerce_product_1[0].name', criteria=None, period=None)], feast_name='my_attribute_view_v1', offline=False, stream_source_name='my_attribute_view_1_stream', entity_key='domain_sessionid')]

This makes the View live, and events will start being processed based on the defined attributes.

# Retreive Attributes from Signals
Now that events are being processed, you can retrieve user attributes from the API:

In [ ]:
response = signals.get_online_attributes(
    source=my_attribute_view,
    identifiers="abc-123",
)

df=response.to_dataframe()
df

,domain_sessionid,products_added_to_cart
0,abc-123,None
